In [ ]:
import os, sys
cwd = os.getcwd()
module_path = os.path.dirname(cwd)  # target working directory

sys.path = [item for item in sys.path if item != module_path]  # remove module_path from sys.path
sys.path.append(module_path)  # add module_path to sys.path

import os
from pathlib import Path
import numpy as np
import time
import librosa
import pandas as pd
import seaborn as sns

from joblib import dump, load

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.utils.fixes import loguniform
from scipy import sparse
from scipy.stats import uniform
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import make_scorer
from pyrcn.metrics import mean_squared_error, accuracy_score
from pyrcn.model_selection import SequentialSearchCV
from pyrcn.util import FeatureExtractor
from pyrcn.datasets import fetch_ptdb_tug_dataset
from pyrcn.echo_state_network import ESNClassifier
from pyrcn.base.blocks import PredefinedWeightsInputToNode, PredefinedWeightsNodeToNode

import matplotlib as mpl
from matplotlib import ticker
import matplotlib.pyplot as plt

sns.set_theme()


mpl.rc('font', **{'family': 'serif'})
%matplotlib inline

import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png', 'pdf')

# TIMIT corpus

In [ ]:
training_sentences = list(Path(r"E:\TIMIT\train").rglob("*.wav"))
test_sentences = list(Path(r"E:\TIMIT\test").rglob("*.wav"))

In [ ]:
len(training_sentences), len(test_sentences)

In [ ]:
def phn_label(phn, frame, hop_length, num_of_frame):
    label = np.empty(num_of_frame, dtype='U5')
    label_number = 0
    idx = int(phn[0][0])
    for i in range(num_of_frame):
        if label_number >= len(phn):
            label[i] = phn[-1][2]
        elif int(phn[label_number][0]) <= idx < int(phn[label_number][1]):
            label[i] = phn[label_number][2]
        else:
            if idx - int(phn[label_number][1]) <= frame / 2:
                label[i] = phn[label_number][2]
                label_number += 1
            else:
                label_number += 1
                label[i] = phn[label_number][2]

        idx += hop_length
    return label


def set_label_number(label):
    phone_39set = {"iy": 0, "ih": 1, "ix": 1, "eh": 2, "ae": 3, "ah": 4, "ax": 4, "ax-h": 4, "uw": 5, "ux": 5, "uh": 6,
                   "aa": 7, "ao": 7, "ey": 8, "ay": 9, "oy": 10, "aw": 11, "ow": 12, "er": 13, "axr": 13,
                   "l": 14, "el": 14, "r": 15, "w": 16, "y": 17, "m": 18, "em": 18, "n": 19, "en": 19, "nx": 19,
                   "ng": 20, "eng": 20, "dx": 21, "jh": 22, "ch": 23, "z": 24, "s": 25, "sh": 26, "zh": 26,
                   "hh": 27, "hv": 27, "v": 28, "f": 29, "dh": 30, "th": 31, "b": 32, "p": 33, "d": 34, "t": 35,
                   "g": 36, "k": 37, "bcl": 38, "pcl": 38, "dcl": 38, "tcl": 38, "gcl": 38, "kcl": 38, "epi": 38,
                   "pau": 38, "h": 38, "q": 38}

    label_idx = np.zeros(len(label))
    for i in range(len(label)):
        label_idx[i] = phone_39set[label[i]]

    return label_idx

In [ ]:
fig, axs = plt.subplots(3, 3)
for k, f in enumerate(shuffle(training_sentences, random_state=42)):
    y, sr = librosa.core.load(str(f), sr=None, mono=False)
    y = librosa.util.normalize(y)
    y = librosa.effects.preemphasis(y)
    mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                         n_mels=40, center=False, window="hamming")
    mel = np.log(mel + 1e-5)
    sns.heatmap(data=mel, ax=axs[np.unravel_index(k, (3, 3))])
    if k >= 8:
        break

In [ ]:
fig, axs = plt.subplots(3, 3)
for k, f in enumerate(shuffle(training_sentences, random_state=42)):
    y, sr = librosa.core.load(str(f), sr=None, mono=False)
    y = librosa.util.normalize(y)
    y = librosa.effects.preemphasis(y)
    mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                         n_mels=40, center=False, window="hamming")
    mel = np.log(mel + 1e-5)
    mel_delta = librosa.feature.delta(mel, width=5, order=1)
    mel_deltadelta = librosa.feature.delta(mel, width=5, order=2)
    sns.heatmap(data=np.vstack((mel, mel_delta, mel_deltadelta)), ax=axs[np.unravel_index(k, (3, 3))])
    if k >= 8:
        break

In [ ]:
fig, axs = plt.subplots(3, 3)
for k, f in enumerate(shuffle(training_sentences, random_state=42)):
    y, sr = librosa.core.load(str(f), sr=None, mono=False)
    y = librosa.util.normalize(y)
    y = librosa.effects.preemphasis(y)
    mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                         n_mels=40, center=False, window="hamming")
    mel = np.log(mel + 1e-5)
    mfcc=librosa.feature.mfcc(sr=sr, S=mel, n_mfcc=13, dct_type=2, n_fft=400,
                              hop_length=160)
    sns.heatmap(data=mfcc, ax=axs[np.unravel_index(k, (3, 3))])
    if k >= 8:
        break

In [ ]:
fig, axs = plt.subplots(3, 3)
for k, f in enumerate(shuffle(training_sentences, random_state=42)):
    y, sr = librosa.core.load(str(f), sr=None, mono=False)
    y = librosa.util.normalize(y)
    y = librosa.effects.preemphasis(y)
    mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                         n_mels=40, center=False, window="hamming")
    mel = np.log(mel + 1e-5)
    mfcc=librosa.feature.mfcc(sr=sr, S=mel, n_mfcc=13, dct_type=2, n_fft=400,
                              hop_length=160)
    mfcc_delta = librosa.feature.delta(mfcc, width=5, order=1)
    mfcc_deltadelta = librosa.feature.delta(mfcc, width=5, order=2)
    sns.heatmap(data=np.vstack((mfcc, mfcc_delta, mfcc_deltadelta)), ax=axs[np.unravel_index(k, (3, 3))])
    if k >= 8:
        break

In [ ]:
scaler = StandardScaler()
X_train = []
y_train = []
X_test = []
y_test = []
from tqdm import tqdm
for k, f in tqdm(enumerate(shuffle(training_sentences, random_state=42)[:100])):
    if not "sa" in str(f):
        y, sr = librosa.core.load(str(f), sr=None, mono=False)
        y = librosa.util.normalize(y)
        y = librosa.effects.preemphasis(y)
        mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                             n_mels=40, center=False, window="hamming")
        mel = np.log(mel + 1e-5)
        mel_delta = librosa.feature.delta(mel, width=5, order=1)
        mel_deltadelta = librosa.feature.delta(mel, width=5, order=2)
        X_train.append(np.vstack((mel, mel_delta, mel_deltadelta)).T)
        scaler.partial_fit(X_train[-1])
        phn = np.loadtxt(str(f).replace(".wav", ".phn"), dtype=str)
        label = phn_label(phn=phn, frame=400, hop_length=160, num_of_frame=X_train[-1].shape[0])
        label_idx = set_label_number(label)
        y_train.append(label_idx)

In [ ]:
for k, f in tqdm(enumerate(shuffle(test_sentences, random_state=42)[:100])):
    if not "sa" in str(f):
        y, sr = librosa.core.load(str(f), sr=None, mono=False)
        y = librosa.util.normalize(y)
        y = librosa.effects.preemphasis(y)
        mel = librosa.feature.melspectrogram(y, sr=sr, n_fft=400, hop_length=160, 
                                             n_mels=40, center=False, window="hamming")
        mel = np.log(mel + 1e-5)
        mel_delta = librosa.feature.delta(mel, width=5, order=1)
        mel_deltadelta = librosa.feature.delta(mel, width=5, order=2)
        X_test.append(np.vstack((mel, mel_delta, mel_deltadelta)).T)
        phn = np.loadtxt(str(f).replace(".wav", ".phn"), dtype=str)
        label = phn_label(phn=phn, frame=400, hop_length=160, num_of_frame=X_test[-1].shape[0])
        label_idx = set_label_number(label)
        y_test.append(label_idx)

In [ ]:
X_train = np.asarray(X_train, dtype=object)
X_test = np.asarray(X_test, dtype=object)
y_train = np.asarray(y_train, dtype=object)
y_test = np.asarray(y_test, dtype=object)
for k, X in enumerate(X_train):
    X_train[k] = scaler.transform(X)
for k, X in enumerate(X_test):
    X_test[k] = scaler.transform(X)

In [ ]:
print(X_train.shape, y_train.shape, X_train[0].shape, y_train[0].shape)

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 1.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bidirectional': False,
                          'alpha': 1e-5,
                          'random_state': 42,
                          'requires_sequence': True}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}
scoring = {"MSE": make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True), 
           "Acc": make_scorer(accuracy_score)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': "MSE"}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': "MSE"}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': "MSE"}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': "MSE"}

searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step3_esn_params, kwargs_step4)]

base_esn = ESNClassifier(**initially_fixed_params)

try:
    sequential_search = load("../../sequential_search_speech_timit_random.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_speech_timit_random.joblib")
print(sequential_search.all_best_params_, sequential_search.all_best_score_)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='viridis', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.55, 0.65)
# sm = plt.cm.ScalarMappable(cmap="viridis")
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
plt.xlim((0, 2.01))
plt.ylim((0, 1.03))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm, label='CER')
fig.set_size_inches(2, 1.25)
ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.savefig('optimize_is_sr_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
plt.xlabel("Leakage")
plt.ylabel("CER")
# plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
# plt.savefig('optimize_leakage_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
plt.xlabel("Alpha")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
plt.grid()
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(sequential_search.best_estimator_.node_to_node.recurrent_weights.todense(), square=True, ax=axs)
fig.set_size_inches(3.2, 2.5)
# plt.savefig('recurrent_weights_random_timit.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(sequential_search.best_estimator_.input_to_node.input_weights.todense().T, ax=axs)
fig.set_size_inches(3.2, 2.5)
# plt.savefig('input_weights_random_timit.pdf', bbox_inches='tight', pad_inches=0)

K-Means training and transition matrix

In [ ]:
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = np.zeros(shape=(n,n))

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M


kmeans = MiniBatchKMeans(n_clusters=50, n_init=200, reassignment_ratio=0, max_no_improvement=50, init='k-means++', verbose=2, random_state=0)
kmeans.fit(X=np.concatenate(np.concatenate((X_train, X_test))))
w_in = np.divide(kmeans.cluster_centers_, np.linalg.norm(kmeans.cluster_centers_, axis=1)[:, None])
input_to_node = PredefinedWeightsInputToNode(predefined_input_weights=w_in.T)
w_rec = transition_matrix(kmeans.labels_)
node_to_node = PredefinedWeightsNodeToNode(predefined_recurrent_weights=w_rec)
node_to_node_eig = PredefinedWeightsNodeToNode(predefined_recurrent_weights=w_rec / np.max(np.abs(np.linalg.eigvals(w_rec))))

# KM-ESN

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 1.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-5,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}
scoring = {"MSE": make_scorer(mean_squared_error, greater_is_better=False, needs_proba=True), 
           "Acc": make_scorer(accuracy_score)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': 'MSE'}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': 'MSE'}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': 'MSE'}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': scoring, 'refit': 'MSE'}

searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step3_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(input_to_node=input_to_node).set_params(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_speech_timit_kmeans.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_speech_timit_kmeans.joblib")
print(sequential_search.all_best_params_, sequential_search.all_best_score_)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='viridis', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.55, 0.65)
# sm = plt.cm.ScalarMappable(cmap="viridis")
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
plt.xlim((0, 2.01))
plt.ylim((0, 1.03))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm, label='CER')
fig.set_size_inches(2, 1.25)
ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.savefig('optimize_is_sr_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
plt.xlabel("Leakage")
plt.ylabel("CER")
# plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
# plt.savefig('optimize_leakage_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
plt.xlabel("Alpha")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(sequential_search.best_estimator_.input_to_node.input_weights.T, ax=axs)
fig.set_size_inches(3.2, 2.5)
# plt.savefig('input_weights_timit_kmeans.pdf', bbox_inches='tight', pad_inches=0)

# KM-ESN with transition matrix

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 1.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-5,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}

searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step3_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(input_to_node=input_to_node,
                                 node_to_node=node_to_node).set_params(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_speech_timit_kmeans_rec.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_speech_timit_kmeans_rec.joblib")
print(sequential_search.all_best_params_, sequential_search.all_best_score_)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='viridis', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.55, 0.65)
# sm = plt.cm.ScalarMappable(cmap="viridis")
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
plt.xlim((0, 2.01))
plt.ylim((0, 1.03))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm, label='CER')
fig.set_size_inches(2, 1.25)
ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.savefig('optimize_is_sr_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
plt.xlabel("Leakage")
plt.ylabel("CER")
# plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
# plt.savefig('optimize_leakage_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
plt.xlabel("Alpha")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(sequential_search.best_estimator_.node_to_node.recurrent_weights, square=True, ax=axs)
fig.set_size_inches(3.2, 1.25)
# plt.savefig('recurrent_weights_timit_transition.pdf', bbox_inches='tight', pad_inches=0)

# KM-ESN with normalized transition matrix

In [ ]:
initially_fixed_params = {'hidden_layer_size': 50,
                          'k_in': 10,
                          'input_scaling': 0.4,
                          'input_activation': 'identity',
                          'bias_scaling': 0.0,
                          'spectral_radius': 1.0,
                          'leakage': 0.1,
                          'k_rec': 10,
                          'reservoir_activation': 'tanh',
                          'bi_directional': False,
                          'wash_out': 0,
                          'continuation': False,
                          'alpha': 1e-5,
                          'random_state': 42}

step1_esn_params = {'input_scaling': uniform(loc=1e-2, scale=1),
                    'spectral_radius': uniform(loc=0, scale=2)}

step2_esn_params = {'leakage': loguniform(1e-5, 1e0)}
step3_esn_params = {'bias_scaling': np.linspace(0.0, 1.0, 11)}
step4_esn_params = {'alpha': loguniform(1e-5, 1e1)}

kwargs_step1 = {'n_iter': 200, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step2 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step3 = {'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}
kwargs_step4 = {'n_iter': 50, 'random_state': 42, 'verbose': 1, 'n_jobs': -1, 'scoring': make_scorer(accuracy_score)}

searches = [('step1', RandomizedSearchCV, step1_esn_params, kwargs_step1),
            ('step2', RandomizedSearchCV, step2_esn_params, kwargs_step2),
            ('step3', GridSearchCV, step3_esn_params, kwargs_step3),
            ('step4', RandomizedSearchCV, step3_esn_params, kwargs_step4)]

base_esn = SeqToSeqESNClassifier(input_to_node=input_to_node,
                                 node_to_node=node_to_node).set_params(**initially_fixed_params)

try:
    sequential_search = load("../sequential_search_speech_timit_kmeans_rec_rho.joblib")
except FileNotFoundError:
    sequential_search = SequentialSearchCV(base_esn, searches=searches).fit(X_train, y_train)
    dump(sequential_search, "../sequential_search_speech_timit_kmeans_rec_rho.joblib")
print(sequential_search.all_best_params_, sequential_search.all_best_score_)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step1"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
ax = sns.scatterplot(x="param_spectral_radius", y="param_input_scaling", hue="mean_test_score", palette='viridis', data=df)
plt.xlabel("Spectral Radius")
plt.ylabel("Input Scaling")

norm = plt.Normalize(0.55, 0.65)
# sm = plt.cm.ScalarMappable(cmap="viridis")
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
sm.set_array([])
plt.xlim((0, 2.01))
plt.ylim((0, 1.03))

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm, label='CER')
fig.set_size_inches(2, 1.25)
ax.yaxis.set_major_locator(ticker.MaxNLocator(5))
ax.xaxis.set_major_locator(ticker.MaxNLocator(5))
# plt.savefig('optimize_is_sr_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step2"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_leakage", y="mean_test_score")
plt.xlabel("Leakage")
plt.ylabel("CER")
# plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.4f'))
# plt.savefig('optimize_leakage_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step3"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_bias_scaling", y="mean_test_score")
plt.xlabel("Bias Scaling")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
df = pd.DataFrame(sequential_search.all_cv_results_["step4"])
df["mean_test_score"] = 1 - df["mean_test_score"]
fig = plt.figure()
fig.set_size_inches(2, 1.25)
ax = sns.lineplot(data=df, x="param_alpha", y="mean_test_score")
plt.xlabel("Alpha")
plt.ylabel("CER")
plt.xlim((0, 1))
tick_locator = ticker.MaxNLocator(5)
ax.xaxis.set_major_locator(tick_locator)
ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.5f'))
# plt.savefig('optimize_bias_scaling_50.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(sequential_search.best_estimator_.node_to_node.recurrent_weights, square=True, ax=axs)
fig.set_size_inches(3.2, 1.25)
# plt.savefig('recurrent_weights_timit_transition_rho.pdf', bbox_inches='tight', pad_inches=0)

In [ ]:
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer().fit(38 - np.concatenate(y_train))

In [ ]:
transition_model = GaussianHMM(n_components=39, 
                               covariance_type="full", 
                               transmat_prior=transition_matrix(38-np.concatenate(y_train).astype(int)),
                               startprob_prior=np.unique(38-np.concatenate(y_train), return_counts=True)[1] / len(np.concatenate(y_train)),
                               n_iter=100).fit(lb.transform(38-np.concatenate(y_train)), lengths=np.asarray([len(y) for y in y_train]))

In [ ]:
fig, axs = plt.subplots(1, 1)

# TODO: Plot the input weights
sns.heatmap(transition_model.transmat_, square=True, ax=axs)
# fig.set_size_inches(3.2, 1.25)

In [ ]:
transition_model.decode(lb.transform(38-y_train[1])), 38-y_train[1]